# 作业2 — 同名消歧
1. 基于key word的尝试
2. 理论上，更多的信息就会更加确定

## 读取文件

In [12]:
words = list()
cs_file_name = "cs-word"
medicel_file_name = "medical-word"
finance_file_name = "finance-word"
file = [cs_file_name, medicel_file_name, finance_file_name]
sample = dict()
start = 0

In [13]:
import random
# 读取文件，构建样本
for file_name in file:
    abstracts = []  # 样本摘要列表
    with open(file_name) as file:
        for line in file.readlines():
            words.extend(line.split())
    # 每个领域生成10条数据
    end = len(words)  # 获取最后一个元素的
    for data in range(100):  # 生成10份摘要
        abstract_generate_index = random.sample(range(start, end), k=10)  # 每份摘要10个单词
        # print(file_name,abstract_generate_index)
        abstracts.append([words[indz] for indz in abstract_generate_index])
    start = end
    # 保存样本
    sample[file_name] = abstracts

In [14]:
print("语料大小:", len(words))

语料大小: 231


In [28]:
print("样本",sample)

样本 {'cs-word': [['GUI', 'Virtual', 'Fault', 'IPC', 'Directory', 'Port', 'Porting', 'Permission', 'Background', 'Portable'], ['Banker', 'Machine', 'Virtual', 'Fault', 'Operating', 'List', 'Process', 'Window', 'Access', 'Deadlock'], ['Ported', 'Background', 'algorithm', 'Thread', 'Memory', 'Time-sharing', 'Boot', 'Control', 'Access', 'IPC'], ['FTP', 'Wrapper', 'Page', 'Process', 'Machine', 'Process', 'Permission', 'IPC', 'Memory', 'Process'], ['Control', 'Banker', 'Window', 'System', 'Access', 'Thread', 'Background', 'GUI', 'Memory', 'Directory'], ['Fault', 'address', 'Control', 'FTP', 'Disk', 'List', 'Permission', 'Memory', 'Process', 'Process'], ['IPC', 'Manager', 'Porting', 'Time-sharing', 'Permission', 'Operating', 'Thread', 'Page', 'Process', 'address'], ['Page', 'Process', 'Workspace', 'GUI', 'Permission', 'Operating', 'Wrapper', 'Manager', 'algorithm', 'Background'], ['FTP', 'Daemon', 'algorithm', 'Access', 'Disk', 'Multithreading', 'Boot', 'Background', 'Port', 'Directory'], ['Di

## 对Key Word进行One Hot编码

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

vecizer = CountVectorizer()
vecizer.fit(words)  # 进行训练

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [17]:
# 把所有样本转化为one hot
word_vectors = []
for vals in sample.values():
    for val in vals:
        # 转化为长句
        val2String = ' '.join(val)
        input = [val2String]
        vect = vecizer.transform(input).toarray()[0]
        word_vectors.append(vect)
print(word_vectors[0:5])

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], dtype=int64), array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

## 观察one hot编码使用距离公式的好坏

In [21]:
from sklearn.metrics.pairwise import euclidean_distances

distance = euclidean_distances([word_vect for word_vect in word_vectors])

import pandas as pd
import numpy as np

dis_df = pd.DataFrame(distance)
dis_df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.000000,4.000000,4.123106,4.690416,3.741657,4.242641,3.872983,3.741657,3.741657,3.464102,...,4.690416,4.472136,4.472136,4.472136,4.472136,4.472136,4.472136,4.472136,4.472136,4.472136
1,4.000000,0.000000,4.358899,4.242641,3.741657,3.741657,4.123106,4.000000,4.242641,3.741657,...,4.690416,4.472136,4.472136,4.472136,4.472136,4.472136,4.472136,4.472136,4.472136,4.472136
2,4.123106,4.358899,0.000000,4.795832,3.316625,4.358899,3.741657,4.123106,3.605551,4.358899,...,4.795832,4.582576,4.358899,4.582576,4.358899,4.582576,4.582576,4.582576,4.582576,4.582576
3,4.690416,4.242641,4.795832,0.000000,4.898979,3.162278,3.872983,3.741657,4.898979,4.242641,...,5.291503,5.099020,5.099020,5.099020,5.099020,5.099020,5.099020,5.099020,5.099020,5.099020
4,3.741657,3.741657,3.316625,4.898979,0.000000,4.242641,4.358899,4.000000,3.741657,4.242641,...,4.690416,4.472136,4.472136,4.472136,4.472136,4.472136,4.472136,4.472136,4.472136,4.472136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,4.472136,4.472136,4.582576,5.099020,4.472136,4.690416,4.582576,4.472136,4.472136,4.472136,...,4.000000,4.242641,3.741657,4.242641,4.242641,0.000000,4.242641,4.242641,4.242641,3.741657
296,4.472136,4.472136,4.582576,5.099020,4.472136,4.690416,4.582576,4.472136,4.472136,4.472136,...,4.242641,4.000000,4.472136,4.000000,4.472136,4.242641,0.000000,4.472136,4.000000,4.000000
297,4.472136,4.472136,4.582576,5.099020,4.472136,4.690416,4.582576,4.472136,4.472136,4.472136,...,4.000000,4.000000,4.472136,4.472136,4.242641,4.242641,4.472136,0.000000,4.242641,4.242641
298,4.472136,4.472136,4.582576,5.099020,4.472136,4.690416,4.582576,4.472136,4.472136,4.472136,...,4.000000,4.472136,4.242641,4.000000,4.242641,4.242641,4.000000,4.242641,0.000000,4.242641


## 寻找最好的EPS和min-sample参数

In [49]:
eps_stat_info = []
for eps in np.arange(start=3.4,stop=5,step=0.1):
     #print("eps:",eps)
     for min_sample in np.arange(start=2,stop=20,step=1):
         #print("min_sample:",min_sample)
         y_pred = DBSCAN(eps=eps,min_samples=min_sample).fit_predict(word_vectors)
         cluster = set([clazz for clazz in y_pred if clazz != -1])
         eps_stat_info.append({'eps': eps, 'n_clusters': cluster,"min_sample":min_sample})
eps_df = pd.DataFrame(eps_stat_info)
# eps_df[eps_df[]]
eps_df[eps_df["n_clusters"]=={0,1,2}]

,eps,n_clusters,min_sample
23,3.5,"{0, 1, 2}",7
41,3.6,"{0, 1, 2}",7
56,3.7,"{0, 1, 2}",4
63,3.7,"{0, 1, 2}",11
72,3.8,"{0, 1, 2}",2
...,...,...,...
139,4.1,"{0, 1, 2}",15
140,4.1,"{0, 1, 2}",16
141,4.1,"{0, 1, 2}",17
142,4.1,"{0, 1, 2}",18


In [42]:
## 使用DBSACN进行聚类操作
from sklearn.cluster import DBSCAN
eps_stat_info = []
y_pred = DBSCAN(eps=3.8, min_samples=5).fit_predict(word_vectors)
print("result:", y_pred)

result: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2]


## 后续：
1. 加入更多信息，如机构信息
2. 思考是否可以使用，使用更短的词向量
3. 使用题目提供的数据集进行进一步的尝试